In [13]:
from bs4 import BeautifulSoup
import requests

In [14]:
base_url = "https://lsgkerala.gov.in"
start_page = base_url + "/en/lbelection/electdistrict/2015/5"
page_data = requests.get(start_page)
soup = BeautifulSoup(page_data.content, "html.parser")

In [15]:
table_section = soup.find(id="block-zircon-content")
table = table_section.find(lambda tag: tag.name=='table') 
rows = table.findAll(lambda tag: tag.name=='tr')

dist_dict = {}
for row in rows[1:-1]:
    dist_name = row.find_all('td')[1].findAll('a')[0].text
    dist_link = row.find_all('td')[1].findAll('a')[0]['href']
    dist_dict[dist_name] = dist_link
print(dist_dict)

{'Thiruvananthapuram': '/en/lbelection/electlbrpt/5/1/2015', 'Kollam': '/en/lbelection/electlbrpt/5/2/2015', 'Pathanamthitta': '/en/lbelection/electlbrpt/5/3/2015', 'Alappuzha': '/en/lbelection/electlbrpt/5/4/2015', 'Kottayam': '/en/lbelection/electlbrpt/5/5/2015', 'Idukki': '/en/lbelection/electlbrpt/5/6/2015', 'Ernakulam': '/en/lbelection/electlbrpt/5/7/2015', 'Thrissur': '/en/lbelection/electlbrpt/5/8/2015', 'Palakkad': '/en/lbelection/electlbrpt/5/9/2015', 'Malappuram': '/en/lbelection/electlbrpt/5/10/2015', 'Kozhikode': '/en/lbelection/electlbrpt/5/11/2015', 'Wayanad': '/en/lbelection/electlbrpt/5/12/2015', 'Kannur': '/en/lbelection/electlbrpt/5/13/2015', 'Kasargode': '/en/lbelection/electlbrpt/5/14/2015'}


In [16]:
dist_soups = {}
for dist in dist_dict:
    dist_page = requests.get(base_url + dist_dict[dist])
    dist_soup = BeautifulSoup(dist_page.content, "html.parser")
    dist_soups[dist] = dist_soup

In [17]:
full_data_dict = {}
for dist in dist_soups:
    table_section = dist_soups[dist].find(id="block-zircon-content")
    table = table_section.find(lambda tag: tag.name=='table') 
    rows = table.findAll(lambda tag: tag.name=='tr')
    panch_dict = {}
    for row in rows[1:-1]:
        panch_name = row.find_all('td')[1].findAll('a')[0].text
        panch_link = row.find_all('td')[1].findAll('a')[0]['href']
        panch_dict[panch_name] = panch_link
    full_data_dict[dist] = panch_dict 
    

In [18]:
for dist in full_data_dict:
    for panch in full_data_dict[dist]:
        panch_page = requests.get(base_url + full_data_dict[dist][panch])
        panch_soup = BeautifulSoup(panch_page.content, "html.parser")
        full_data_dict[dist][panch] = panch_soup

In [22]:
def extract_ruling_party(panch_soup):
    ruling_party = None
    table_section = panch_soup.find(id="block-zircon-content")
    table = table_section.find(lambda tag: tag.name=='table') 
    rows = table.findAll(lambda tag: tag.name=='tr')
    party_dict = {}
    for row in rows:
        tds = row.findAll('td')
        if not len(tds) > 0:
            continue
        party = tds[4].text
        party_dict[party] = party_dict.get(party, 0) + 1
    if(len(party_dict) > 0):
        ruling_party = max(party_dict, key=party_dict.get)
    return ruling_party

In [27]:
full_data_dict_copy = full_data_dict.copy()
for dist in full_data_dict_copy:
    for panch in full_data_dict_copy[dist]:
        panch_soup = full_data_dict_copy[dist][panch]
        if(isinstance(panch_soup, str)):
            continue
        ruling_party = extract_ruling_party(panch_soup)
        full_data_dict_copy[dist][panch] = ruling_party

full_data_dict_copy

{'Thiruvananthapuram': {'Chemmaruthy': 'CPI(M)',
  'Edava': 'CPI(M)',
  'Elakamon': 'CPI(M)',
  'Manamboor': 'INC',
  'Ottoor': 'INC',
  'Cherunniyoor': 'CPI(M)',
  'Vettoor': 'INC',
  'Kilimanoor': 'CPI(M)',
  'Pazhayakunnumel': 'CPI(M)',
  'Karavaram': 'CPI(M)',
  'Madavoor': 'INC',
  'Pallickal': 'CPI(M)',
  'Nagaroor': 'CPI(M)',
  'Navaikulam': 'INC',
  'Pulimath': 'CPI(M)',
  'Azhoor': 'CPI(M)',
  'Anchuthengu': 'INC',
  'Vakkom': 'CPI(M)',
  'Chirayinkeezhu': 'CPI(M)',
  'Kizhuvilam': 'INC',
  'Mudakkal': 'INC',
  'Kadakkavoor': 'CPI(M)',
  'Kallara': 'INC',
  'Nellanad': 'INC',
  'Pullampara': 'CPI(M)',
  'Vamanapuram': 'CPI(M)',
  'Pangode': 'INC',
  'Nanniyode': 'CPI(M)',
  'Peringammala': 'CPI(M)',
  'Manickal': 'CPI(M)',
  'Aryanad': 'INC',
  'Poovachal': 'CPI(M)',
  'Vellanad': 'INC',
  'Vithura': 'INC',
  'Uzhamalakkal': 'CPI(M)',
  'Kuttichal': 'CPI(M)',
  'Tholicode': 'CPI(M)',
  'Kattakada': 'INC',
  'Anad': 'INC',
  'Aruvikkara': 'CPI(M)',
  'Panavoor': 'CPI(M)',
  'Ka

In [26]:
isinstance(full_data_dict_copy['Alappuzha']['Venmoney'], str)


False

In [28]:
import csv
with open('2015_party.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    for dist in full_data_dict_copy:
        for panch in full_data_dict_copy[dist]:
            ruling_party = full_data_dict_copy[dist][panch]
            row = (dist, panch, ruling_party)
            csvwriter.writerow(row)